In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [2]:
url = 'https://www.the-numbers.com/movie/budgets/all/501'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [3]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">501</td>
<td><a href="/box-office-chart/daily/2004/07/07">Jul 7, 2004</a></td>
<td><b><a href="/movie/King-Arthur-(2004)#tab=summary">King Arthur</a></b></td>
<td class="data"> $90,000,000</td>
<td class="data"> $51,877,963</td>
<td class="data"> $203,653,524</td>
</tr>
<tr><td class="data">502</td>
<td><a href="/box-office-chart/daily/1997/04/25">Apr 25, 1997</a></td>
<td><b><a href="/movie/Volcano#tab=summary">Volcano</a></b></td>
<td class="data"> $90,000,000</td>
<td class="data"> $47,546,796</td>
<td class="data"> $120,100,000</td>
</tr>
<tr><td class="data">503</td>
<td><a href="/box-office-chart/daily/1997/03/26">Mar 26, 1997</a></td>
<td><b><a href="/movie/Devils-Own-The#tab=summary">The Devil's Own</a></b></td>
<td class="data"> $90,000,000</td>
<td class="data"> $42,885,593</td>
<td class="data"> $140,900,

In [4]:
rows = [row for row in table.find_all('tr')]

In [5]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Jul 7, 2004': ['/box-office-chart/daily/2004/07/07',
  '501',
  'Jul 7, 2004',
  'King Arthur',
  '\xa0$90,000,000',
  '\xa0$51,877,963',
  '\xa0$203,653,524'],
 'Apr 25, 1997': ['/box-office-chart/daily/1997/04/25',
  '502',
  'Apr 25, 1997',
  'Volcano',
  '\xa0$90,000,000',
  '\xa0$47,546,796',
  '\xa0$120,100,000'],
 'Mar 26, 1997': ['/box-office-chart/daily/1997/03/26',
  '503',
  'Mar 26, 1997',
  "The Devil's Own",
  '\xa0$90,000,000',
  '\xa0$42,885,593',
  '\xa0$140,900,000'],
 'Jul 19, 2002': ['/box-office-chart/daily/2002/07/19',
  '504',
  'Jul 19, 2002',
  'K-19: The Widowmaker',
  '\xa0$90,000,000',
  '\xa0$35,168,966',
  '\xa0$65,716,126'],
 'Aug 19, 2011': ['/box-office-chart/daily/2011/08/19',
  '505',
  'Aug 19, 2011',
  'Conan the Barbarian',
  '\xa0$90,000,000',
  '\xa0$21,295,021',
  '\xa0$63,356,133'],
 'Oct 21, 2011': ['/box-office-chart/daily/2011/10/21',
  '506',
  'Oct 21, 2011',
  'The Three Musketeers',
  '\xa0$90,000,000',
  '\xa0$20,377,913',
  '\xa0$139

In [6]:
budget6 = pd.DataFrame(movies).T  #transpose
budget6.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget6.shape

(99, 7)

In [7]:
budget6['budget'] = budget6['budget'].str.replace('$', '')
budget6['budget'] = budget6['budget'].str.replace(',', '')
budget6['budget'] = budget6['budget'].astype(int)

In [9]:
budget6.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Jul 7, 2004",/box-office-chart/daily/2004/07/07,501,"Jul 7, 2004",King Arthur,90000000,"$51,877,963","$203,653,524"
"Apr 25, 1997",/box-office-chart/daily/1997/04/25,502,"Apr 25, 1997",Volcano,90000000,"$47,546,796","$120,100,000"
"Mar 26, 1997",/box-office-chart/daily/1997/03/26,503,"Mar 26, 1997",The Devil's Own,90000000,"$42,885,593","$140,900,000"
"Jul 19, 2002",/box-office-chart/daily/2002/07/19,504,"Jul 19, 2002",K-19: The Widowmaker,90000000,"$35,168,966","$65,716,126"
"Aug 19, 2011",/box-office-chart/daily/2011/08/19,505,"Aug 19, 2011",Conan the Barbarian,90000000,"$21,295,021","$63,356,133"


In [10]:
budget6.to_csv('budget6.csv')